TODO
Correction for SepReformerSettings sr duplicated

In [1]:
import os
import warnings
import torch

In [2]:
warnings.filterwarnings('ignore')

In [3]:
workdir, _ = os.path.split(os.path.abspath(os.getcwd()))
input = os.path.join(os.getcwd(), 'SepReformer/sample_wav/sample_WSJ.wav')

In [4]:
from simple_vg.vg_types import AudioSetting

settings: AudioSetting = {
    'mono': True,
    'n_fft': None,
    'sr': 16000,
    'use_torch': True,
    'window_size': None,
    'opt_settings': None
}


In [5]:
from simple_vg.process_audio import LoadAudioFile, LoadAudioSettings, LoadAudioBackends

load_settings: LoadAudioSettings = {
    'backend': LoadAudioBackends.TORCH_SOX,
    'effects': [],
    'channel_first': True
}

settings['opt_settings'] = load_settings

load_audio = LoadAudioFile(settings)

In [6]:
from simple_vg.simple_elements import ClosureElement

only_wav = ClosureElement(lambda _, inp: inp[0], None)

In [7]:
from simple_vg.speech_separation.sepreformer import *

sep_model_settings: SepReformerSetting = {
    'batch_size': 1,
    'chunk_max_len': None,
    'distributed_gpu': False,
    'model': SepReformerModels.SepReformer_Base_WSJ0,
    'n_speaker': 2,
    'sr': 16000
}


In [8]:
from simple_vg.process_overlap import *

sep_settings: SeparateOverlappedSettings = {
    'model_settings': sep_model_settings,
    'model': SeparateOverlappedModels.SepReformer
}

overlap_settings: ProcessOverlappedAudioSettings = {
    'tasks': ProcessOverlappedAudioTasks.SEPARATE_AUDIO,
    'separation_settings': sep_settings
}

settings['opt_settings'] = overlap_settings
overlap_elem = ProcessOverlappedAudio(settings)

In [9]:
from simple_vg.activity_detection import *

vad_p_settings: VADPipelineSetting = {
    'vad_model': VADModels.Silero_VAD
}

vad_settings: VoiceActivityDetectionSetting = {
    'pipeline_settings': vad_p_settings,
    'tasks': [
        VoiceActivityDetectionTasks.ACTIVITY_DETECTION, 
        VoiceActivityDetectionTasks.SPLIT_BY_ACTIVITY
    ]
}

settings['opt_settings'] = vad_settings

vad_elem = VoiceActivityDetection(settings)

In [10]:
import simple_vg.extract_text as stt
from simple_vg.extract_text import ExtractText

stt_settings = stt.default_settings()
stt_settings['batch_size'] = 2
settings['opt_settings'] = stt_settings
stt_mod = ExtractText(settings) #Input shape: (..., S) [(x, y, z, w, S) => view(x*y*z*w, S)]


In [11]:
import sys
from simple_vg.pipeline import VGPipeline

pipeline_elems = [
    load_audio, 
    only_wav, 
    overlap_elem,
    stt_mod]
pipeline = VGPipeline()
pipeline.sequential(pipeline_elems)
pipeline.run(input=input, dbg=True)


Index-0 LoadAudioFile Pipeline Output: tuple, shape: None
Index-1 ClosureElement Pipeline Output: Tensor, shape: torch.Size([1, 147186])
Index-2 ProcessOverlappedAudio Pipeline Output: Tensor, shape: torch.Size([1, 2, 1, 147184])
Index-3 ExtractText Pipeline Output: list, shape: None


In [12]:
stt_mod.model.dataset.indice

[[0, 0, 919], [1, 0, 919]]

In [13]:
stt_mod.model.result

[[DecodingResult(audio_features=tensor([[ 0.0347,  0.3894, -0.3850,  ..., -0.0782, -0.6201,  0.2014],
          [-0.0539, -0.0062, -0.1096,  ...,  0.2605,  0.6401, -0.2522],
          [-0.2634, -0.1658, -0.1598,  ...,  0.0565,  0.6421,  0.6436],
          ...,
          [-0.1322, -0.3555, -0.0687,  ..., -0.1731, -0.0271,  0.0078],
          [-0.2720, -0.0267, -0.4807,  ..., -0.3486, -0.2411, -0.3455],
          [-0.2585,  0.0527, -0.5142,  ..., -0.6338,  0.0499,  0.2527]],
         device='cuda:0', dtype=torch.float16), language='en', language_probs=None, tokens=[50365, 4162, 264, 624, 13, 50, 13, 13346, 575, 38942, 42893, 294, 5162, 3259, 11, 415, 8194, 572, 643, 337, 264, 12380, 26049, 281, 5300, 1179, 6846, 281, 2371, 264, 7241, 13, 50824], text='Since the U.S. currency has strengthened sharply in recent weeks, he sees no need for the Federal Reserve to raise interest rates to protect the dollar.', avg_logprob=-0.03825973763185389, no_speech_prob=1.1966396840437654e-12, temperature=